## Error Handling

In [ ]:
CREATE TABLE IF NOT EXISTS foo(a INT PRIMARY KEY, question VARCHAR NOT NULL);
INSERT INTO foo(a, question) VALUES (42, 'The answer is 42.');

In [ ]:
CREATE OR REPLACE FUNCTION oops(a INT, question VARCHAR)
    RETURNS VOID
    LANGUAGE plpgsql
AS $$
BEGIN
    INSERT INTO foo(a, question) VALUES (oops.a, oops.question);
END $$;

SELECT oops(42, 'Same answer?');

In [ ]:
CREATE OR REPLACE FUNCTION oops(a INT, question VARCHAR)
    RETURNS VOID
    LANGUAGE plpgsql
AS $$
BEGIN
    INSERT INTO foo(a, question) VALUES (oops.a, oops.question);
EXCEPTION WHEN unique_violation THEN
    RAISE NOTICE 'Key "%" already exists in table.', oops.a;
END $$;

SELECT oops(42, 'Same answer?');

---
### Handling errors in plpgsql

```SQL
[ <<label>> ]
[ DECLARE
    declarations ]
BEGIN
    statements
EXCEPTION
    WHEN condition [ OR condition ... ] THEN
        handler_statements
    [ WHEN condition [ OR condition ... ] THEN
          handler_statements
      ... ]
END;
```

---
### Integrity Constraint Violations 
- See https://www.postgresql.org/docs/current/errcodes-appendix.html

SQLSTATE|Condition
--------|---------
23000|integrity_constraint_violation
23001|restrict_violation
23502|not_null_violation
23503|foreign_key_violation
23505|unique_violation
23514|check_violation
23P01|exclusion_violation

---
### Getting error information
    GET STACKED DIAGNOSTICS variable { = | := } item [ , ... ];
### Items

Name|Description
----|-----------
RETURNED_SQLSTATE|the SQLSTATE error code of the exception
COLUMN_NAME|the name of the column related to exception
CONSTRAINT_NAME|the name of the constraint related to exception
PG_DATATYPE_NAME|the name of the data type related to exception
MESSAGE_TEXT|the text of the exception's primary message
TABLE_NAME|the name of the table related to exception
SCHEMA_NAME|the name of the schema related to exception
PG_EXCEPTION_DETAIL|the text of the exception's detail message, if any
PG_EXCEPTION_HINT|the text of the exception's hint message, if any
PG_EXCEPTION_CONTEXT|line(s) of text describing the call stack at the time of the exception


In [ ]:
CREATE OR REPLACE FUNCTION oops(a INT, question VARCHAR)
    RETURNS VOID
    LANGUAGE plpgsql
AS $$
DECLARE
    msg text;
    stt text;
    tbl text;
    sch text;
    cst text;
BEGIN
    INSERT INTO foo(a, question) VALUES (oops.a, oops.question);
EXCEPTION 
    WHEN OTHERS THEN
        GET STACKED DIAGNOSTICS 
            msg := MESSAGE_TEXT,
            stt := RETURNED_SQLSTATE,
            tbl := TABLE_NAME,
            sch := SCHEMA_NAME,
            cst := CONSTRAINT_NAME;
        RAISE NOTICE 'SQLSTATE=%, Table=%.%, Constraint=%', stt, sch, tbl, cst;
        RAISE;
END $$;

SELECT oops(42, 'Same answer?');

---
### Clean up

In [ ]:
DROP FUNCTION IF EXISTS oops(INT, VARCHAR);
DROP TABLE IF EXISTS foo;